## Sentinel-2 feature tracking
### ethz-02-02-02

This application takes a pair of Sentinel-2 products and performs feature tracking using the run_dic package

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Sentinel-2 feature tracking'),
                ('abstract', 'Sentinel-2 feature tracking'),
                ('id', 'ewf-ethz-02-02-02')])

In [ ]:
band = dict([('id', 'band'),
             ('value', 'B08'),
             ('title', 'band'),
             ('abstract', 'desired band to analyze'),
             ('options', 'B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12')])

In [ ]:
window_size = dict([('id', 'window_size'),
                    ('title', 'window_size'),
                    ('abstract', 'window size in pixels'),
                    ('value', '512')])

In [ ]:
oversampling_factor = dict([('id', 'oversampling_factor'),
                            ('title', 'oversampling_factor'),
                            ('abstract', 'oversampling factor'),
                            ('value', '2')])

In [ ]:
pixel_size = dict([('id', 'pixel_size'),
                   ('title', 'pixel_size'),
                   ('abstract', 'pixel size in meters'),
                   ('value', '10')])

In [ ]:
color_scale_limits = dict([('id', 'color_scale_limits'),
                           ('title', 'color_scale_limits'),
                           ('abstract', 'color_scale_limits'),
                           ('value', '0,10')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [ ]:
input_identifiers = ('S2A_MSIL2A_20190601T163311_N0212_R140_T15MYV_20190601T221326',
                     'S2A_MSIL2A_20190611T163311_N0212_R140_T15MYV_20190611T222739')

**Input references**

These are the Sentinel-1 catalogue references

In [ ]:
input_references = ('https://catalog.terradue.com:443/sentinel2/search?uid=S2A_MSIL2A_20190601T163311_N0212_R140_T15MYV_20190601T221326',
                    'https://catalog.terradue.com:443/sentinel2/search?uid=S2A_MSIL2A_20190611T163311_N0212_R140_T15MYV_20190611T222739')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
from ellip_snap_helpers import create_metadata
import xml.etree.ElementTree as ET

import geopandas as gpd
from datetime import datetime

import matplotlib.pyplot as plt

from shapely.geos import ReadingError
import gdal
import lxml.etree as etree

import warnings
warnings.filterwarnings("ignore")

import glob

sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import subprocess

import cioppy
ciop = cioppy.Cioppy()

## Read the products

### check if all the products have the same track number

In [ ]:
metadata = dict()

try:
    search0 = ciop.search(end_point=input_references[0],
                         params=dict(),
                         output_fields='identifier, startdate, enddate, wkt',
                         model='GeoTime')[0]
    
    search1 = ciop.search(end_point=input_references[1],
                         params=dict(),
                         output_fields='identifier, startdate, enddate, wkt',
                         model='GeoTime')[0]
    
    if search0['startdate'] > search1['startdate']:
        master_date = search1['startdate']
        slave_date = search0['startdate']
    else:
        master_date = search0['startdate']
        slave_date = search1['startdate']
        
    metadata['startdate'] = master_date
    metadata['enddate'] = slave_date
    metadata['wkt'] = search0['wkt']
    
    print metadata

except Exception as e:
    print('ERROR: could not retrieve products metadata. {}'.format(e))

#### Read the products

In [ ]:
band_dict = {'B01':'60m', 
             'B02':'10m', 
             'B03':'10m', 
             'B04':'10m', 
             'B05':'20m', 
             'B06':'20m', 
             'B07':'20m', 
             'B08':'10m', 
             'B8A':'20m', 
             'B09':'20m', 
             'B11':'20m', 
             'B12':'20m'}

In [ ]:
for root, _, files in os.walk(os.path.join(data_path,input_identifiers[0],input_identifiers[0]+'.SAFE')):
    for file in files:
        if '{}_{}.jp2'.format(band['value'], band_dict[band['value']]) in file:
            path1 = os.path.join(root, file)
            
print path1

In [ ]:
for root, dirs,files in os.walk(os.path.join(data_path,input_identifiers[1],input_identifiers[1]+'.SAFE')):
    for file in files:
        if '{}_{}.jp2'.format(band['value'], band_dict[band['value']]) in file:
            path2 = os.path.join(root, file)
            
print path2

### run_dic

#### writing the input_dic.txt

In [ ]:
with open('input_dic.txt', 'wb') as file:
    file.write('{}\n'.format(path1))
    file.write('{}\n'.format(path2))
    file.write('{} {} {}\n'.format(window_size['value'],oversampling_factor['value'], pixel_size['value']))
    file.write('{} {}\n'.format(color_scale_limits['value'].split(',')[0], color_scale_limits['value'].split(',')[1]))

In [ ]:
with open('input_dic.txt') as file:
    print file.read()

#### running the package 

In [ ]:
os.environ['LD_LIBRARY_PATH'] = '/opt/v94/runtime/glnxa64:/opt/v94/bin/glnxa64:/opt/v94/sys/os/glnxa64:/opt/v94/extern/bin/glnxa64'

command = 'import run_dic; mr = run_dic.initialize(); mr.run_dic(\"input_dic.txt\", nargout=0)'

options = ['python', '-c', command]

p = subprocess.Popen(options,
                     stdout=subprocess.PIPE,
                     stdin=subprocess.PIPE,
                     stderr=subprocess.PIPE)

res, err = p.communicate()

if res:
    print 'RESULTS:\n'
    for el in res.split('\n'):
        print el

if err:
    print 'ERRORS:\n'
    for el in res.split('\n'):
        print el

In [ ]:
os.remove('input_dic.txt')

In [ ]:
src = gdal.Open(path1)

geo_transform = src.GetGeoTransform()
projection = src.GetProjection()

src.FlushCache()

In [ ]:
trf = {'geo_transform' : geo_transform,
        'projection' : projection}

with open('geo_transform_projection.txt', 'wb') as file:
    file.write(str(trf))
    file.close()

In [ ]:
with open('geo_transform_projection.txt') as f:
    print f.read()

In [ ]:
for file in os.listdir('./'):
    if '.txt' in file:
        print file
        #print gdal.Info(file)
        #with rasterio.open(file, 'r') as ds:
        #    arr = ds.read()
            
        #drv = gdal.GetDriverByName('GTiff')
        #ds = drv.Create('{}_{}.tif'.format(os.path.splitext(os.path.basename(file))[0], band['value']), arr.shape[2], arr.shape[1], arr.shape[0], gdal.GDT_Byte)
        #ds.SetGeoTransform(geo_transform)
        #ds.SetProjection(projection)
        #for band_number in range(arr.shape[0]):
        #    ds.GetRasterBand(band_number+1).WriteArray(arr[band_number])
        #ds.FlushCache()
        
        metadata['identifier'] = '{}_{}'.format(os.path.splitext(os.path.basename(file))[0], band['value'])
        metadata['title'] = metadata['identifier']
        create_metadata(metadata, metadata['identifier'])

In [ ]:
os.listdir('./')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.